# Identifier les coordonnées GPS à partir des pixels d'une image

Le but : filtrer les zones éligibles à l'installation d'éoliennes. Les données étant trop lourdes à visualiser, nous allons essayer de déterminer les coordonnées à partir des pixels pour ne charger qu'un array à la fin.

Pour que le code fonctionne, faire un screenshot de la carte en plaçant une ville repère 1 dans le coin supérieur gauche et une ville repère 2 dans le coin inférieur droit.
Ensuite préciser les coordonnées de ces villes dans les constantes `city_top_left` et `city_bottom_right`. (Clic droit sur la ville dans Google Maps pour copier les coordonnées)

In [33]:
from PIL import Image
import numpy as np
import pandas as pd
import plotly.express as px

Quelques variables. Ne pas modifier les constantes car elles correspondent aux enjeux.

Ajuster le nom de la région et les coordonnées.

In [34]:
COLOR_FILTER_01 = [129, 1, 121]
COLOR_FILTER_02 = [208, 134, 255]

region = "ILE DE FRANCE"
city_top_left = [49.44388360753829, 1.0963138947193751]
city_bottom_right = [47.325755534083456, 5.0683452608213075]

Préciser ici le chemin de l'image à analyser.

In [35]:
image_path = "../src/image3.png"
image = np.array(Image.open(image_path))

width, height, _ = image.shape

(width, height)

(587, 718)

In [66]:
lat1, lon1 = city_top_left
lat2, lon2 = city_bottom_right

# compression, etirement de la grille
# ce sont des valeurs trouvees a tatons, il faudra surement les ajuster
lon_compression_coeff = 0.8
lat_compression_coeff = 1.25


def get_gps_coordinates(x: int, y: int, width: int, height: int,
                        lat1: float, lon1: float, lat2: float, lon2: float
                        ) -> tuple[float, float]:
    """
    Calcule les coordonnees GPS pour un pixel donné.
    
    x, y : Position du pixel dans l'image
    width, height : Dimensions de l'image
    lat1, lon1 : Coordonnees GPS du coin superieur gauche
    lat2, lon2 : Coordonnees GPS du coin inférieur droit
    """
    latitude = lat1 + (y / height) * (lat2 - lat1) * lat_compression_coeff
    longitude = lon1 + (x / width) * (lon2 - lon1) * lon_compression_coeff

    return latitude, longitude

# # example
# middle_x, middle_y = width // 2, height // 2
# middle_lat, middle_lon = get_gps_coordinates(middle_x, middle_y, width, height, lat2, lon2, lat1, lon1)

# middle_lat, middle_lon


In [72]:
coordinates_list = []

for y in range(height):
    for x in range(width):
        for color in [COLOR_FILTER_01, COLOR_FILTER_02]:
            try:
                if np.array_equal(image[y, x][:3], color):  # [:3] skip alpha
                    lat, lon = get_gps_coordinates(x, y,
                                                   width, height,
                                                   lat1, lon1,
                                                   lat2, lon2)

                    color = f"rgb({color[0]},{color[1]},{color[2]})"
                    coordinates_list.append([y, x, lat, lon, color])
            except Exception as e:
                pass  # n'arrive pas a calculer car len = len-1 je crois

df = pd.DataFrame(coordinates_list, columns=["Y", "X",
                                             "Latitude", "Longitude",
                                             "Color"])

df.head()

,Y,X,Latitude,Longitude,Color
0,93,359,49.100942,3.039700,"rgb(208,134,255)"
1,93,360,49.100942,3.045113,"rgb(208,134,255)"
2,93,361,49.100942,3.050526,"rgb(208,134,255)"
3,94,359,49.097254,3.039700,"rgb(208,134,255)"
4,94,368,49.097254,3.088420,"rgb(129,1,121)"


In [74]:
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude",
                        hover_data=["X", "Y"],
                        color="Color",
                        zoom=7,
                        height=800)


fig.update_layout(mapbox_style="open-street-map")

fig.show()